##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# テンプレートを使用して TFX パイプラインを作成する

注意: このチュートリアルは、Google Cloud Vertex AI Workbench で実行することをお勧めします。[このノートブックを Vertex AI Workbenchで起動](https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?q=download_url%3Dhttps%253A%252F%252Fraw.githubusercontent.com%252Ftensorflow%252Ftfx%252Fmaster%252Fdocs%252Ftutorials%252Ftfx%252Ftemplate.ipynb)してください。

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td><a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/template"> <img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で表示</a></td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/template.ipynb"> <img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tfx/tutorials/tfx/template.ipynb"> <img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a></td>
<td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tfx/tutorials/tfx/template.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a></td>
</table></div>

## はじめに

このドキュメントでは、TFX Python パッケージで提供される*テンプレート*を使用し、TensorFlow Extended (TFX) パイプラインを作成する手順について説明します。命令の多くは Linux シェルコマンドであり、AI Platform Notebooks インスタンスで実行されます。ここでは `!` を使用してこれらのコマンドの呼び出しに対応する Jupyter Notebook コードセルが提供されています。

シカゴ市が公開した[タクシー乗降データセット](https://data.cityofchicago.org/Transportation/Taxi-Trips/wrvz-psew)を使用してパイプラインを構築します。このパイプラインをベースラインとして利用して、データセットを使用して独自のパイプラインを構築してみることを強くお勧めします。


## ステップ 1. 環境をセットアップする

AI Platform Pipeline は、パイプラインを構築するための開発環境と、新しく構築されたパイプラインを実行するための Kubeflow Pipeline クラスターを準備します。

**注意：**特定の TensorFlow バージョン、または GPU インスタンスを選択するには、AI Platform Notebook に TensorFlow にプリインストールされたインスタンスを作成します。


`kfp` の追加要件を使用して `tfx` python パッケージをインストールします。

In [ ]:
import sys
# Use the latest version of pip.
!pip install --upgrade pip
# Install tfx and kfp Python packages.
!pip install --upgrade "tfx[kfp]<2"

TFX のバージョンを確認します。

In [ ]:
!python3 -c "from tfx import version ; print('TFX version: {}'.format(version.__version__))"

AI Platform Pipeline では、TFX は [Kubeflow Pipelines](https://www.kubeflow.org/docs/pipelines/overview/pipelines-overview/) を使用してホストされた Kubernetes 環境で実行されます。

Kubeflow パイプラインを使用するためにいくつかの環境変数を設定しましょう。

まず、GCP プロジェクト ID を取得します。

In [ ]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GOOGLE_CLOUD_PROJECT=shell_output[0]
%env GOOGLE_CLOUD_PROJECT={GOOGLE_CLOUD_PROJECT}
print("GCP project ID:" + GOOGLE_CLOUD_PROJECT)

また、KFP クラスターにアクセスする必要があります。Google Cloud Console の [AI Platform]&gt; [Pipeline] メニューからアクセスできます。KFP クラスターの「エンドポイント」は、パイプラインダッシュボードの URL から見つけることができます。または、このノートブックを起動した [はじめに] ページの URL から取得することもできます。`ENDPOINT` 環境変数を作成し、それを KFP クラスターエンドポイントに設定します。**ENDPOINT には、URL のホスト名部分のみを含める必要があります。**たとえば、KFP ダッシュボードの URL が `https://1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com/#/start` の場合、ENDPOINT 値は `1e9deb537390ca22-dot-asia-east1.pipelines.googleusercontent.com` になります。

> **注意：以下の ENDPOINT 値を設定する必要があります。**

In [ ]:
# This refers to the KFP cluster endpoint
ENDPOINT='' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

現在の GCP プロジェクトで、イメージ名を `tfx-pipeline` と設定します。

In [ ]:
# Docker image name for the pipeline image.
CUSTOM_TFX_IMAGE='gcr.io/' + GOOGLE_CLOUD_PROJECT + '/tfx-pipeline'

以上でパイプラインを作成する準備ができました。

## ステップ 2. 事前定義されたテンプレートをプロジェクトディレクトリにコピーする

このステップでは、事前定義されたテンプレートから追加のファイルをコピーして、作業パイプラインプロジェクトディレクトリとファイルを作成します。

以下の `PIPELINE_NAME` を変更することで、パイプラインに別の名前を付けることができます。また、これはファイルが配置されるプロジェクトディレクトリ名にもなります。

In [ ]:
PIPELINE_NAME="my_pipeline"
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"),"imported",PIPELINE_NAME)

TFX には、TFX python パッケージと `taxi` テンプレートが含まれています。分類や回帰など、ポイントワイズの予測問題を解決するには、このテンプレートを利用してスタートできます。

`tfx template copy` CLI コマンドは、事前定義されたテンプレートファイルをプロジェクトディレクトリにコピーします。

In [ ]:
!tfx template copy \
  --pipeline-name={PIPELINE_NAME} \
  --destination-path={PROJECT_DIR} \
  --model=taxi

このノートブックの作業ディレクトリコンテキストをプロジェクトディレクトリに変更します。

In [ ]:
%cd {PROJECT_DIR}

> 注意: 作成されたプロジェクトディレクトリをクリックして、左側の `File Browser` のディレクトリを変更することを忘れないでください。

## ステップ 3. コピーしたソースファイルを閲覧する

TFX テンプレートは、Python ソースコード、サンプルデータ、パイプラインの出力を分析する Jupyter Notebook など、パイプラインを構築するための基本的なスキャフォールドファイルを提供します。`taxi` テンプレートは、[Airflow Tutorial ](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop)と同じ*シカゴタクシー*データセットと ML モデルを使用します。

ここでは、各 Python ファイルを簡単に紹介します。

- `pipeline` - このディレクトリには、パイプラインの定義が含まれています。
    - `configs.py` — パイプライン ランナーの共通定数を定義します。
    - `pipeline.py` — TFX コンポーネントとパイプラインを定義します。
- `models` - このディレクトリには、機械学習モデルの定義が含まれています。
    - `features.py`、`features_test.py` — モデルの特徴を定義します。
    - `preprocessing.py`、`preprocessing_test.py` — `tf::Transform`を使用して前処理ジョブを定義します。
    - `estimator` - このディレクトリには、Estimator ベースのモデルが含まれています。
        - `constants.py` — モデルの定数を定義します。
        - `model.py`、`model_test.py` — TF estimator を使用して DNN モデルを定義します
    - `keras` - このディレクトリには、Keras ベースのモデルが含まれています。
        - `constants.py` — モデルの定数を定義します。
        - `model.py`、`model_test.py` — Keras を使用して DNN モデルを定義します。
- `beam_runner.py`、`kubeflow_runner.py` — オーケストレーション エンジンごとにランナーを定義します。


名前に `_test.py` が含まれているファイルがあることに気が付きましたか。これらはパイプラインの単体テストであり、独自のパイプラインを実装するときに単体テストを追加することをお勧めします。テストファイルのモジュール名に `-m` フラグを指定すると、単体テストを実行できます。通常、モジュール名を取得するには、`.py` 拡張子を削除し、`/` を `.` に置き換えます。以下に例を示します。

In [ ]:
!{sys.executable} -m models.features_test
!{sys.executable} -m models.keras.model_test


## ステップ 4. 最初の TFX パイプラインを実行する

TFX パイプラインのコンポーネントは、実行ごとに出力を [ML メタデータアーティファクト](https://www.tensorflow.org/tfx/guide/mlmd)として生成するため、それらをどこかに保存する必要があります。KFP クラスターがアクセスできる任意のストレージを使用できます。この例では、Google Cloud Storage (GCS) を使用します。デフォルトの GCS バケットが自動的に作成されているはずです。その名前は `<your-project-id>-kubeflowpipelines-default` になります。


サンプルデータを GCS バケットにアップロードして、後でパイプラインで使用できるようにします。

In [ ]:
!gsutil cp data/data.csv gs://{GOOGLE_CLOUD_PROJECT}-kubeflowpipelines-default/tfx-template/data/taxi/data.csv

`tfx pipeline create` コマンドを使用して TFX パイプラインを作成します。

> 注意: KFP のパイプラインを作成するときは、パイプラインの実行に使用されるコンテナイメージが必要です。そして、`skaffold` がイメージを構築します。skaffold は Docker ハブからベースイメージをプルするため、最初にイメージをビルドするときは 5〜10 分かかりますが、2 回目以降のビルドにはそれほど時間がかかりません。

In [ ]:
!tfx pipeline create  --pipeline-path=kubeflow_runner.py --endpoint={ENDPOINT} \
--build-image

パイプラインの作成中に、Docker イメージをビルドするための `Dockerfile` が生成されます。これらのファイルを他のソースファイルと一緒にソース管理システム (git など) に追加することを忘れないでください。

注意: `airflow` がインストールされておらず、`--engine` が指定されていない場合、`kubeflow` がオーケストレーションエンジンとして自動的に選択されます。

次に、`tfx run create` コマンドを使用して、新しく作成されたパイプラインで実行を開始します。

In [ ]:
!tfx run create --pipeline-name={PIPELINE_NAME} --endpoint={ENDPOINT}

または、KFP ダッシュボードでパイプラインを実行することもできます。新しい実行は、KFP ダッシュボードの[試験]の下に一覧表示されます。試験をクリックすると、進行状況を監視し、実行中に作成されたアーティファクトを視覚化できます。

ただし、KFP ダッシュボードにアクセスすることをお勧めします。KFP ダッシュボードには、Google Cloud Console の Cloud AI Platform Pipelines メニューからアクセスできます。ダッシュボードにアクセスすると、パイプラインを見つけて、パイプラインに関する豊富な情報にアクセスできます。たとえば、*試験*メニューで実行を見つけることができ、試験で実行を開くと、*アーティファクト*メニューでパイプラインからすべてのアーティファクトを表示できます。

> 注意: パイプラインの実行に失敗した場合は、KFP ダッシュボードの[試験]タブで各 TFX コンポーネントの詳細なログを確認できます。

失敗の主な原因の 1 つは、許可関連の問題です。KFP クラスタに Google Cloud API へのアクセス権限があることを確認してください。これは、[GCP で KFP クラスタを作成するときに](https://cloud.google.com/ai-platform/pipelines/docs/setting-up)構成します。[GCP でのトラブルシューティング ドキュメント](https://cloud.google.com/ai-platform/pipelines/docs/troubleshooting)を参照してください。

## ステップ 5. データ検証用のコンポーネントを追加する

このステップでは、`StatisticsGen`、`SchemaGen`、および、`ExampleValidator` などのデータ検証用のコンポーネントを追加します。データ検証については、[Tensorflow データ検証の基礎](https://www.tensorflow.org/tfx/data_validation/get_started)を参照してください。

> **ダブルクリックしてディレクトリを `pipeline` に変更し、もう一度ダブルクリックして  `pipeline.py`を開きます**。`StatisticsGen`、`SchemaGen` および `ExampleValidator` をパイプラインに追加する 3 行を見つけてコメントを外します。（ヒント：`TODO(step 5):` を含むコメントを検索します）。編集後は、必ず `pipeline.py` を保存してください。

次に、パイプライン定義を変更して既存のパイプラインを更新します。`tfx pipeline update` コマンドを使用してパイプラインを更新し、続いて `tfx run create` コマンドを使用して更新されたパイプラインの新しい実行を作成します。


In [ ]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
# You can run the pipeline the same way.
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

### パイプライン出力を確認する

KFP ダッシュボードにアクセスし、パイプライン実行のページでパイプライン出力を見つけます。左側の*{nbsp}[試験] *タブをクリックし、[試験] ページの*{nbsp}[すべての実行] *をクリックします。パイプラインの名前の実行が表示されるはずです。

## ステップ 6. トレーニング用のコンポーネントを追加する

このステップでは、`Transform`、`Trainer`、`Resolver`、`Evaluator`、および、`Pusher` などのトレーニングとモデル検証用のコンポーネントを追加します。

> **ダブルクリックして `pipeline.py` を開きます**。`Transform`、`Trainer`、`Resolver`、`Evaluator` および `Pusher` をパイプラインに追加する 5 行を見つけてコメントを外します (ヒント: `TODO(step 6):` を見つけます。)

以前と同様に、変更されたパイプライン定義で既存のパイプラインを更新する必要があります。手順はステップ 5 と同じです。`tfx pipeline update` を使用してパイプラインを更新し、`tfx run create` を使用して実行を作成します。


In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

この実行が正常に終了すると、AI プラットフォームパイプラインで最初の TFX パイプラインが作成され、実行されます。

**注意：**モデルコードを変更した場合は、コンテナイメージも再構築する必要があります。`pipeline update` コマンドの `--build-image` フラグを使用して再構築をトリガーできます。

**注意：**パイプライン実行を作成するたびに、入力とパラメーターが変更されていなくても、すべてのコンポーネントが何度も実行されることに気付いたかもしれません。これは時間とリソースの無駄なので、パイプラインキャッシングを使用してこれらの実行をスキップできます。`pipeline.py` の `Pipeline` オブジェクトに `enable_cache=True` を指定することで、キャッシュを有効にできます。


## ステップ 7. (*オプション*) BigQueryExampleGen を試してみる

[BigQuery](https://cloud.google.com/bigquery) は、サーバーレスでスケーラビリティと費用対効果の高いクラウド データ ウェアハウスです。BigQuery は、TFX のトレーニング サンプルのソースとして使用できます。このステップでは、パイプラインに `BigQueryExampleGen` を追加します。

> **`pipeline.py` をダブルクリックして開きます**。`CsvExampleGen` をコメントアウトし、`BigQuery Example Gen` のインスタンスを作成する行のコメントを外します。また、`create_pipeline` 関数の`query` 引数のコメントも外す必要があります。

BigQuery に使用する GCP プロジェクトを指定する必要があります。そのためには、パイプラインの作成時に `beam_pipeline_args` に `--project` を設定します。

> **`configs.py`** をダブルクリックして開きます。`BIG_QUERY_WITH_DIRECT_RUNNER_BEAM_PIPELINE_ARGS` と `BIG_QUERY_QUERY` の定義のコメントを外します。このファイルのリージョンの値を、GCP プロジェクトの正しい値に置き換えます。

> **注意: 続行する前に、GCP リージョンを `configs.py` ファイルに設定する必要があります。**

> **ディレクトリを 1 レベル上に変更します。**ファイル リストの上にあるディレクトリの名前をクリックします。ディレクトリ名はパイプライン名で、パイプライン名を変更しなかった場合は `my_pipeline` です。

> **ダブルクリックして `kubeflow_runner.py`** を開きます。`create_pipeline` 関数の 2 つの引数  `query` と `beam_pipeline_args` のコメントを外します。

パイプラインで BigQuery をサンプル ソースとして使用する準備ができました。前と同じようにパイプラインを更新し、ステップ 5 と 6 で行ったように新しい実行を作成します。

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

## ステップ 8. (*オプション*) KFP で Dataflow を試してみる

いくつかの [TFX コンポーネントは Apache Beam を使用](https://www.tensorflow.org/tfx/guide/beam)してデータ並列パイプラインを実装します。そのため、[Google Cloud Dataflow](https://cloud.google.com/dataflow/) を使用してデータ処理ワークロードを分散できます。このステップでは、Apache Beam のデータ処理バックエンドとして Dataflow を使用するように Kubeflow オーケストレーターを設定します。

> **`pipeline` をダブルクリックしてディレクトリを変更し、`configs.py`** をダブルクリックして開きます。`GOOGLE_CLOUD_REGION` と `DATAFLOW_BEAM_PIPELINE_ARGS` の定義のコメントを外します。

> **ディレクトリを 1 レベル上に変更します。**ファイル リストの上にあるディレクトリの名前をクリックします。ディレクトリ名はパイプライン名で、変更しなかった場合は `my_pipeline` です。

> **`kubeflow_runner.py`** をダブルクリックして開きます。`beam_pipeline_args` のコメントを外します。（ステップ 7 で追加した最新の`beam_pipeline_args` も必ずコメントアウトしてください）。

以上で、パイプラインで Dataflow を使用する準備が整いました。ステップ 5 と 6 で行ったように、パイプラインを更新し、実行を作成します。

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

Cloud Console の Dataflow で Dataflow のジョブを見つけることができます。


## ステップ 9. (*オプション*) TKFP で Cloud AI Platform のトレーニングと予測を試してみる

TFX は、[トレーニングと予測のための Cloud AI Platform](https://cloud.google.com/ai-platform/) など、いくつかのマネージド GCP サービスと相互運用します。機械学習モデルをトレーニングするためのマネージド サービスである Cloud AI Platform Training を使用するように `Trainer` コンポーネントを設定できます。さらに、モデルが構築され、サービングする準備ができたら、サービングするためにモデルを Cloud AI Platform Prediction に *push* できます。このステップでは、Cloud AI Platform サービスを使用するように `Trainer`  コンポーネントと`Pusher` コンポーネントを設定します。

> ファイルを編集する前に、最初に *AI Platform Training と Prediction API* を有効にする必要がある場合があります。

> **`pipeline` をダブルクリックしてディレクトリを変更し、ダブルクリックして `configs.py`** を開きます。`GOOGLE_CLOUD_REGION`、`GCP_AI_PLATFORM_TRAINING_ARGS`、`GCP_AI_PLATFORM_SERVING_ARGS` の定義のコメントを外します。カスタム ビルドのコンテナ イメージを使用して Cloud AI Platform Training でモデルをトレーニングするため、`GCP_AI_PLATFORM_TRAINING_ARGS` の `masterConfig.imageUri` を上記の `CUSTOM_TFX_IMAGE` と同じ値に設定する必要があります。

> **ディレクトリを 1 レベル上に変更し、ダブルクリックして `kubeflow_runner.py`** を開きます。`ai_platform_training_args` と `ai_platform_serving_args` のコメントを外します。

ステップ  5 と 6 で行ったように、パイプラインを更新し、実行を作成します。

In [ ]:
!tfx pipeline update \
--pipeline-path=kubeflow_runner.py \
--endpoint={ENDPOINT}
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

トレーニング ジョブは [Cloud AI Platform ジョブ](https://console.cloud.google.com/ai-platform/jobs)で見つけることができます。パイプラインが正常に完了した場合、[Cloud AI Platform モデル](https://console.cloud.google.com/ai-platform/models)でモデルを見つけることができます。

## ステップ 10. データをパイプラインに取り込む

シカゴタクシーデータセットを使用してモデルのパイプラインを作成しました。次に、データをパイプラインに入れます。

データは、GCS や BigQuery など、パイプラインがアクセスできる場所であればどこにでも保存できます。データにアクセスするには、パイプライン定義を変更する必要があります。

1. データがファイルに保存されている場合は、`kubeflow_runner.py` または `local_runner.py` の `DATA_PATH` を変更し、ファイルの場所に設定します。データが BigQuery に保存されている場合は、`pipeline/configs.py` の `BIG_QUERY_QUERY` を変更して、データを正しくクエリします。
2. `models`/<code>features.py</code> に特徴量を追加します。
3. `models`/<code>preprocessing.py</code> を変更して、<a>トレーニング用の入力データを変換</a>します。
4. `models/keras/model.py` と`models/keras/constants.py` を変更して、[機械学習モデルを記述](https://www.tensorflow.org/tfx/guide/trainer)します。
    - Estimator ベースのモデルを使用することもできます。 `pipeline/configs.py` の `RUN_FN` 定数を `models.estimator.model.run_fn` に変更します。

詳細については、[トレーナーコンポーネントガイド](https://www.tensorflow.org/tfx/guide/trainer)を参照してください。

## クリーンアップ

このプロジェクトで使用されているすべての Google Cloud リソースをクリーンアップするには、チュートリアルで使用した [Google Cloud プロジェクトを削除](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)します。

または、各コンソールにアクセスして、個々のリソースをクリーンアップすることもできます。

- [Google Cloud Storage](https://console.cloud.google.com/storage)
- [Google Container Registry](https://console.cloud.google.com/gcr)
- [Google Kubernetes Engine](https://console.cloud.google.com/kubernetes)
